# Bayesian Optimisation Notebook

<h4> This notebook compartmentalises the Bayesian Optimisation aspect of this project for cleanliness</h4>

In [17]:
import numpy as np
import torch
import torch.nn
import torch.optim 
from torch.utils.data import DataLoader, TensorDataset
import os
from tqdm import tqdm
import random

import numpy as np
import optuna
import json

# my custom modules
import data_utils as du
import nn_tools as nt
import model_eval as me

os.chdir('/Users/Mak/Desktop/Imperial College London/Year Four/MSci Project/Codebase/')
DEVICE = torch.device("mps")

import importlib

importlib.reload(me)
importlib.reload(nt)
importlib.reload(du)


<module 'data_utils' from '/Users/Mak/Desktop/Imperial College London/Year Four/MSci Project/Codebase/data_utils.py'>

Setting the data and storage paths - to extract the raw data and store the binned, normalised data appropriately

In [20]:
normal_path = 'nn_outputs/wilson_csr'
raw_path = 'raw_data/dstore_csr'
number_bins = 10
test_amt = 768

In [ ]:
du.file_to_normalise_4d(raw_path, normal_path, number_bins, bkg=False)

In [21]:
train_inputs, train_targets, test_inputs, test_targets, file_names = du.load_minmax_heights(normal_path, test_amt)

In [24]:
cfg_CVAE = {'batch_train' : 64,
       'batch_test' : 8,
       'epochs' : 150,
       'dim': [len(train_inputs[0]), len(train_targets[0])],
       }

stopping_patience = 10

def objective(trial, train_inputs=train_inputs, train_targets=train_targets,test_inputs=test_inputs, test_targets=test_targets):

    s = 42

    torch.use_deterministic_algorithms(True)
    torch.backends.mps.deterministic = True
    torch.backends.mps.benchmark = False

    np.random.seed(s)
    torch.manual_seed(s)
    torch.mps.manual_seed(s)
    random.seed(s)

    # Choose the number of layers on either end
    n_encoder_layers = trial.suggest_int('n_encoder_layers', 1, 4)
    n_decoder_layers = trial.suggest_int('n_decoder_layers', 1, 4)

    # Choose the size of each layer in the model
    encoder_layers = [trial.suggest_int(f'encoder_layer_{i}', 48, 512) for i in range(n_encoder_layers)]
    decoder_layers = [trial.suggest_int(f'decoder_layer_{i}', 48, 1024) for i in range(n_decoder_layers)]
    
    # Choose whether to include a dropouts layer or not
    dropout_enc = [trial.suggest_categorical(f'dropout_enc_{i}', [True, False]) for i in range(n_encoder_layers)]
    dropout_dec = [trial.suggest_categorical(f'dropout_dec_{i}', [True, False]) for i in range(n_decoder_layers)]

    # Defined the all important latent dimension size
    latent_dim = trial.suggest_int('latent_dim', 8, 64)
    
    # Initialise dataloaders and model with the trial parameters
    load_train = DataLoader(TensorDataset(train_inputs, train_targets), batch_size=64, shuffle=True, num_workers=0)
    load_test = DataLoader(TensorDataset(test_inputs, test_targets), batch_size=8, shuffle=False, num_workers=0)

    model = nt.CVAE_Opt(input_shape=cfg_CVAE['dim'][0], output_shape=cfg_CVAE['dim'][1], latent_dim=latent_dim, encoder_layers=encoder_layers, decoder_layers=decoder_layers,
                 dropout_enc=dropout_enc, dropout_dec=dropout_dec)
    model.to(DEVICE)
    optimiser = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)


    early_stopping_patience = 10
    best_val_loss = np.inf
    epochs_without_improvement = 0

    # Training loops
    for epoch in (pbar := tqdm(range(cfg_CVAE['epochs']))):
        avg_loss, avg_val_loss = nt.train_epoch_early_CVAE(model, load_train, load_test, nt.loss_function, optimiser, DEVICE)
        pbar.set_description(f"Training Epoch: {epoch + 1}/{cfg_CVAE['epochs']}, Avg. Loss: {avg_loss:.4f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_without_improvement = 0

        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= early_stopping_patience:
            print(f"Early stopping triggered after {epoch + 1} epochs.")
            break

    # Evaluate the model finally, and return the test loss to be minimised
    test_loss = nt.test_epoch_CVAE(model=model, dataloader=load_test, criterion=nt.loss_function, device=DEVICE)

    return test_loss

class EarlyStoppingCallback:
    def __init__(self, patience):
        self.patience = patience
        self.best_value = np.inf
        self.epochs_without_improvement = 0

    def __call__(self, study, trial):
        current_value = trial.value
        if current_value < self.best_value:
            self.best_value = current_value
            self.epochs_without_improvement = 0
        else:
            self.epochs_without_improvement += 1

        if self.epochs_without_improvement >= self.patience:
            print(f"Early stopping triggered after {trial.number + 1} trials.")
            study.stop()

# Create the Optuna study with early stopping
DEVICE = torch.device('mps')
early_stopping = EarlyStoppingCallback(patience=stopping_patience)
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=75, callbacks=[early_stopping])


[I 2025-03-11 15:21:49,814] A new study created in memory with name: no-name-f7ed276d-f444-4269-962b-fa18005fe627
Training Epoch: 34/150, Avg. Loss: 0.0020:  22%|██▏       | 33/150 [00:13<00:47,  2.47it/s]
[I 2025-03-11 15:22:03,610] Trial 0 finished with value: 0.001917514884553384 and parameters: {'n_encoder_layers': 4, 'n_decoder_layers': 2, 'encoder_layer_0': 427, 'encoder_layer_1': 196, 'encoder_layer_2': 459, 'encoder_layer_3': 475, 'decoder_layer_0': 617, 'decoder_layer_1': 538, 'dropout_enc_0': True, 'dropout_enc_1': True, 'dropout_enc_2': False, 'dropout_enc_3': False, 'dropout_dec_0': True, 'dropout_dec_1': False, 'latent_dim': 26}. Best is trial 0 with value: 0.001917514884553384.


Early stopping triggered after 34 epochs.


Training Epoch: 68/150, Avg. Loss: 0.0019:  45%|████▍     | 67/150 [00:20<00:25,  3.30it/s]
[I 2025-03-11 15:22:24,028] Trial 1 finished with value: 0.0018995304344571196 and parameters: {'n_encoder_layers': 4, 'n_decoder_layers': 1, 'encoder_layer_0': 87, 'encoder_layer_1': 470, 'encoder_layer_2': 225, 'encoder_layer_3': 145, 'decoder_layer_0': 382, 'dropout_enc_0': False, 'dropout_enc_1': False, 'dropout_enc_2': False, 'dropout_enc_3': False, 'dropout_dec_0': False, 'latent_dim': 31}. Best is trial 1 with value: 0.0018995304344571196.


Early stopping triggered after 68 epochs.


Training Epoch: 28/150, Avg. Loss: 0.0020:  18%|█▊        | 27/150 [00:08<00:40,  3.07it/s]
[I 2025-03-11 15:22:32,921] Trial 2 finished with value: 0.0019689061264216434 and parameters: {'n_encoder_layers': 3, 'n_decoder_layers': 2, 'encoder_layer_0': 241, 'encoder_layer_1': 503, 'encoder_layer_2': 455, 'decoder_layer_0': 70, 'decoder_layer_1': 153, 'dropout_enc_0': True, 'dropout_enc_1': False, 'dropout_enc_2': False, 'dropout_dec_0': False, 'dropout_dec_1': False, 'latent_dim': 43}. Best is trial 1 with value: 0.0018995304344571196.


...

Early stopping triggered after 80 epochs.


Training Epoch: 44/150, Avg. Loss: 0.0020:  29%|██▊       | 43/150 [00:17<00:43,  2.45it/s]
[I 2025-03-11 15:33:04,807] Trial 33 finished with value: 0.001919640002597589 and parameters: {'n_encoder_layers': 3, 'n_decoder_layers': 4, 'encoder_layer_0': 113, 'encoder_layer_1': 210, 'encoder_layer_2': 194, 'decoder_layer_0': 432, 'decoder_layer_1': 538, 'decoder_layer_2': 539, 'decoder_layer_3': 402, 'dropout_enc_0': False, 'dropout_enc_1': True, 'dropout_enc_2': True, 'dropout_dec_0': False, 'dropout_dec_1': True, 'dropout_dec_2': True, 'dropout_dec_3': True, 'latent_dim': 57}. Best is trial 24 with value: 0.0018901043331425171.


Early stopping triggered after 44 epochs.


Training Epoch: 45/150, Avg. Loss: 0.0020:  29%|██▉       | 44/150 [00:15<00:38,  2.79it/s]
[I 2025-03-11 15:33:20,730] Trial 34 finished with value: 0.0019184036027581897 and parameters: {'n_encoder_layers': 2, 'n_decoder_layers': 4, 'encoder_layer_0': 236, 'encoder_layer_1': 249, 'decoder_layer_0': 734, 'decoder_layer_1': 603, 'decoder_layer_2': 948, 'decoder_layer_3': 158, 'dropout_enc_0': False, 'dropout_enc_1': True, 'dropout_dec_0': False, 'dropout_dec_1': True, 'dropout_dec_2': True, 'dropout_dec_3': True, 'latent_dim': 60}. Best is trial 24 with value: 0.0018901043331425171.


Early stopping triggered after 45 epochs.
Early stopping triggered after 35 trials.


In [25]:
model_param = study.best_trial
output_file_path = normal_path.split('/')[1] + '.json'

with open(output_file_path, 'w') as json_file:
    json.dump(model_param.params, json_file, indent=4)

print(f"Output dictionary saved to {output_file_path}")

Output dictionary saved to wilson_csr.json
